# Advanced Practice: Special Methods (__repr__, __str__, __eq__)

This notebook contains advanced practice problems on special methods, focused on:

* __repr__ vs __str__
* Custom equality with __eq__
* Identity vs equality (is vs ==)
* Good API design and best practices for these methods

Each problem is followed by a reference solution.

Try to solve the problems yourself before looking at the solutions.


## Problem 1 – Designing __repr__ and __str__ for a 2D Vector

Design a `Vector2D` class with the following requirements:

1. It stores two floats: `x` and `y`.
2. Implement:
   * `__repr__` so that it is unambiguous and ideally evaluable, e.g. `Vector2D(x=1.5, y=-2.0)`.
   * `__str__` so that it is user-friendly, e.g. `<1.5, -2.0>`.
3. Implement a method `length()` that returns the Euclidean length of the vector.
4. Show that:
   * Printing a list of vectors uses `__repr__`.
   * Printing a single vector with `print(v)` uses `__str__`.

Best practice hint:
`__repr__` is primarily for developers (debugging, logging, sometimes copy-paste into code),
`__str__` is for human-friendly display.


In [1]:
# === Your work for Problem 1 goes here ===
# Implement Vector2D so that the tests below pass.

import math

class Vector2D:
    pass  # TODO: remove this and implement your solution

# --- Basic tests (feel free to add more) ---
# v = Vector2D(3.0, 4.0)
# print("__repr__ :", repr(v))   # should look like Vector2D(x=3.0, y=4.0)
# print("__str__  :", str(v))    # should look like <3.0, 4.0>
# print("length   :", v.length())  # should be 5.0

# vectors = [Vector2D(1, 2), Vector2D(-0.5, 0)]
# print("list of vectors:", vectors)  # uses __repr__ for each element


### Solution – Problem 1


In [2]:
import math

class Vector2D:
    def __init__(self, x: float, y: float) -> None:
        self.x = float(x)
        self.y = float(y)

    def length(self) -> float:
        return math.hypot(self.x, self.y)

    def __repr__(self) -> str:
        # Developer/debug representation: unambiguous and close to valid Python
        # code that could recreate the object.
        return f"Vector2D(x={self.x!r}, y={self.y!r})"

    def __str__(self) -> str:
        # User-facing representation: concise and friendly.
        return f"<{self.x}, {self.y}>"

# Demonstration
v = Vector2D(3.0, 4.0)
print("__repr__ :", repr(v))
print("__str__  :", str(v))
print("length   :", v.length())

vectors = [Vector2D(1, 2), Vector2D(-0.5, 0)]
print("list of vectors:", vectors)


__repr__ : Vector2D(x=3.0, y=4.0)
__str__  : <3.0, 4.0>
length   : 5.0
list of vectors: [Vector2D(x=1.0, y=2.0), Vector2D(x=-0.5, y=0.0)]


## Problem 2 – Equality and Normalization (__eq__)

Implement a `Polynomial` class representing a polynomial in one variable, for example:

    p(x) = 2x^2 + 3x + 1

Use a coefficient list representation:

* index = power of x
* value = coefficient

For example:

* `Polynomial([1, 3, 2])` represents `1 + 3x + 2x^2`.

Requirements:

1. Store coefficients in a list internally (e.g. `self._coeffs`).
2. Implement:
   * `__repr__` like `Polynomial(coeffs=[1, 3, 2])`.
   * `__str__` that renders a human-readable polynomial, e.g. `2x^2 + 3x + 1`.
     * Skip zero terms.
     * Respect signs (`-` vs `+`).
3. Implement `__eq__` so that mathematically equal polynomials compare equal,
   even if trailing zeros differ in the internal representation:
   * `Polynomial([1, 0, 2]) == Polynomial([1, 0, 2, 0, 0])` should be `True`.
4. If `other` is not a `Polynomial`, `__eq__` should return `NotImplemented`.

Best practice hint:
Returning `NotImplemented` lets Python try reversed comparisons and keeps equality
behaviour consistent for other types.


In [3]:
# === Your work for Problem 2 goes here ===

class Polynomial:
    pass  # TODO: remove this and implement your solution

# --- Basic tests ---
# p1 = Polynomial([1, 0, 2])         # 1 + 0x + 2x^2
# p2 = Polynomial([1, 0, 2, 0, 0])   # same polynomial, padded with zeros
# print("p1:", p1)
# print("p2:", p2)
# print("p1 == p2 ?", p1 == p2)      # should be True

# p3 = Polynomial([0, 1])            # x
# p4 = Polynomial([1, 0])            # 1
# print("p3:", p3)
# print("p4:", p4)
# print("p3 == p4 ?", p3 == p4)      # should be False

# print("repr(p1):", repr(p1))


### Solution – Problem 2


In [4]:
class Polynomial:
    def __init__(self, coeffs):
        # Make a defensive copy
        self._coeffs = list(coeffs)
        self._strip_trailing_zeros()

    def _strip_trailing_zeros(self) -> None:
        # Remove redundant zeros at the high-degree end.
        while len(self._coeffs) > 1 and self._coeffs[-1] == 0:
            self._coeffs.pop()

    @property
    def coeffs(self):
        # Expose an immutable view (tuple) to avoid accidental modification.
        return tuple(self._coeffs)

    def __repr__(self) -> str:
        return f"Polynomial(coeffs={list(self._coeffs)!r})"

    def __str__(self) -> str:
        if all(c == 0 for c in self._coeffs):
            return "0"

        terms = []
        for power, coeff in enumerate(self._coeffs):
            if coeff == 0:
                continue

            sign = "-" if coeff < 0 else "+"
            abs_coeff = abs(coeff)

            if power == 0:
                term_mag = str(abs_coeff)
            elif power == 1:
                if abs_coeff == 1:
                    term_mag = "x"
                else:
                    term_mag = f"{abs_coeff}x"
            else:
                if abs_coeff == 1:
                    term_mag = f"x^{power}"
                else:
                    term_mag = f"{abs_coeff}x^{power}"
            terms.append((sign, term_mag))

        # First term keeps its sign only if it's negative
        sign, term_mag = terms[0]
        if sign == "-":
            result = "-" + term_mag
        else:
            result = term_mag

        # Remaining terms: always add sign + space + magnitude
        for sign, term_mag in terms[1:]:
            result += f" {sign} {term_mag}"

        return result

    def __eq__(self, other) -> bool:
        if not isinstance(other, Polynomial):
            return NotImplemented
        return self.coeffs == other.coeffs

# Demonstration
p1 = Polynomial([1, 0, 2])
p2 = Polynomial([1, 0, 2, 0, 0])
print("p1:", p1)
print("p2:", p2)
print("p1 == p2 ?", p1 == p2)

p3 = Polynomial([0, 1])
p4 = Polynomial([1, 0])
print("p3:", p3)
print("p4:", p4)
print("p3 == p4 ?", p3 == p4)

print("repr(p1):", repr(p1))


p1: 1 + 2x^2
p2: 1 + 2x^2
p1 == p2 ? True
p3: x
p4: 1
p3 == p4 ? False
repr(p1): Polynomial(coeffs=[1, 0, 2])


## Problem 3 – Tagged Values and Type-Safe Equality

You are designing a small system to attach tags to values, for example:

* "user_id" with value `42`
* "temperature" with value `18.5`

Implement a `TaggedValue` class with:

1. Attributes:
   * `tag` (string)
   * `value` (any Python object)
2. Implement:
   * `__repr__` as `TaggedValue(tag='user_id', value=42)` (or similar).
3. Implement `__eq__` with these rules:
   * Two `TaggedValue` objects are equal if both their tags and values are equal.
   * A `TaggedValue` should compare equal to a plain value (non-`TaggedValue`) only if:
     * The tag is "value" and `self.value == other`.
   * If `other` is of an unsupported type, return `NotImplemented`.

Examples:

    TaggedValue("user_id", 1) == TaggedValue("user_id", 1)   # True
    TaggedValue("user_id", 1) == TaggedValue("user_id", 2)   # False
    TaggedValue("user_id", 1) == TaggedValue("order_id", 1)  # False
    TaggedValue("value", 10) == 10                            # True
    TaggedValue("user_id", 10) == 10                          # False

4. Show that comparisons are symmetric in the "value" case:

    TaggedValue("value", 10) == 10      # True
    10 == TaggedValue("value", 10)      # True

Best practice hints:
* Implement `__eq__` defensively and return `NotImplemented` for unknown types.
* Avoid writing asymmetric equality semantics by accident.


In [5]:
# === Your work for Problem 3 goes here ===

class TaggedValue:
    pass  # TODO: remove this and implement your solution

# --- Basic tests ---
# tv1 = TaggedValue("user_id", 1)
# tv2 = TaggedValue("user_id", 1)
# tv3 = TaggedValue("user_id", 2)
# tv4 = TaggedValue("order_id", 1)

# print("tv1 == tv2:", tv1 == tv2)  # True
# print("tv1 == tv3:", tv1 == tv3)  # False
# print("tv1 == tv4:", tv1 == tv4)  # False

# tv_plain_ok = TaggedValue("value", 10)
# tv_plain_bad = TaggedValue("user_id", 10)

# print("tv_plain_ok == 10:", tv_plain_ok == 10)     # True
# print("10 == tv_plain_ok:", 10 == tv_plain_ok)     # True
# print("tv_plain_bad == 10:", tv_plain_bad == 10)   # False
# print("10 == tv_plain_bad:", 10 == tv_plain_bad)   # False

# print("repr(tv1):", repr(tv1))


### Solution – Problem 3


In [6]:
class TaggedValue:
    def __init__(self, tag: str, value):
        self.tag = str(tag)
        self.value = value

    def __repr__(self) -> str:
        return f"TaggedValue(tag={self.tag!r}, value={self.value!r})"

    def __eq__(self, other) -> bool:
        # TaggedValue vs TaggedValue
        if isinstance(other, TaggedValue):
            return self.tag == other.tag and self.value == other.value

        # TaggedValue vs plain value: only allowed if tag == "value"
        if self.tag == "value":
            return self.value == other

        # Unsupported comparison – let Python try other.__eq__ or fall back
        return NotImplemented

# Demonstration
tv1 = TaggedValue("user_id", 1)
tv2 = TaggedValue("user_id", 1)
tv3 = TaggedValue("user_id", 2)
tv4 = TaggedValue("order_id", 1)

print("tv1 == tv2:", tv1 == tv2)
print("tv1 == tv3:", tv1 == tv3)
print("tv1 == tv4:", tv1 == tv4)

tv_plain_ok = TaggedValue("value", 10)
tv_plain_bad = TaggedValue("user_id", 10)

print("tv_plain_ok == 10:", tv_plain_ok == 10)
print("10 == tv_plain_ok:", 10 == tv_plain_ok)
print("tv_plain_bad == 10:", tv_plain_bad == 10)
print("10 == tv_plain_bad:", 10 == tv_plain_bad)

print("repr(tv1):", repr(tv1))


tv1 == tv2: True
tv1 == tv3: False
tv1 == tv4: False
tv_plain_ok == 10: True
10 == tv_plain_ok: True
tv_plain_bad == 10: False
10 == tv_plain_bad: False
repr(tv1): TaggedValue(tag='user_id', value=1)


## Problem 4 – Debugging a Broken __eq__ Implementation

You are given the following buggy class:

    class Circle:
        def __init__(self, radius):
            self.radius = radius

        def __repr__(self):
            return f"Circle(radius={self.radius})"

        def __eq__(self, other):
            # BUGGY IMPLEMENTATION
            return self.radius == other.radius

Tasks:

1. Explain in a markdown cell what is wrong with this `__eq__` implementation.
   Consider what happens with `Circle(1) == 1`.
2. Fix the implementation so that:
   * Only `Circle` instances are considered comparable.
   * For other types, `__eq__` returns `NotImplemented`.
3. Add a few tests that demonstrate:
   * `Circle(1) == Circle(1)` is `True`.
   * `Circle(1) == Circle(2)` is `False`.
   * `Circle(1) == 1` is `False`, and `1 == Circle(1)` is also `False` (no exceptions).


### Your explanation – Problem 4

(Write your explanation here before looking at the reference answer.)


In [7]:
# === Your fixed Circle implementation for Problem 4 ===

class Circle:
    def __init__(self, radius: float):
        self.radius = float(radius)

    def __repr__(self) -> str:
        return f"Circle(radius={self.radius})"

    def __eq__(self, other) -> bool:
        # TODO: implement robust equality here
        return NotImplemented

# --- Your tests ---
c1 = Circle(1)
c2 = Circle(1)
c3 = Circle(2)

print("c1 == c2:", c1 == c2)  # True
print("c1 == c3:", c1 == c3)  # False
print("c1 == 1:", c1 == 1)    # False
print("1 == c1:", 1 == c1)    # False


c1 == c2: False
c1 == c3: False
c1 == 1: False
1 == c1: False


### Reference explanation – Problem 4

The buggy implementation:

    def __eq__(self, other):
        return self.radius == other.radius

has several issues:

1. It assumes that `other` has a `.radius` attribute. This is not true for arbitrary objects like `1`,
   strings, or unrelated classes, and will raise `AttributeError`.
2. It does not check that `other` is a `Circle` (or a compatible type). Equality should be type-aware.
3. It never returns `NotImplemented`, which is the correct way to signal that the comparison does not
   make sense for the given type.

A robust implementation should check `isinstance(other, Circle)` and return `NotImplemented` when `other`
is not a `Circle`.


### Solution – Problem 4


In [8]:
class Circle:
    def __init__(self, radius: float):
        self.radius = float(radius)

    def __repr__(self) -> str:
        return f"Circle(radius={self.radius})"

    def __eq__(self, other) -> bool:
        if not isinstance(other, Circle):
            return NotImplemented
        return self.radius == other.radius

# Demonstration
c1 = Circle(1)
c2 = Circle(1)
c3 = Circle(2)

print("c1:", c1)
print("c2:", c2)
print("c3:", c3)

print("c1 == c2:", c1 == c2)
print("c1 == c3:", c1 == c3)
print("c1 == 1:", c1 == 1)     # False
print("1 == c1:", 1 == c1)     # False


c1: Circle(radius=1.0)
c2: Circle(radius=1.0)
c3: Circle(radius=2.0)
c1 == c2: True
c1 == c3: False
c1 == 1: False
1 == c1: False


## Problem 5 – Identity vs Equality (is vs ==)

Consider the following `User` class:

    class User:
        def __init__(self, username, email):
            self.username = username
            self.email = email

1. Implement `__repr__` so instances display as `User(username='alice', email='alice@example.com')`.
2. Implement `__eq__` with the following semantics:
   * Users are considered equal if they have the same `username` (email is ignored for equality).
   * Only compare with other `User` instances. For anything else, return `NotImplemented`.
3. Demonstrate the difference between `is` and `==` by:
   * Creating two different `User` objects with the same username and email.
   * Showing that:
     * `u1 is u2` is `False`.
     * `u1 == u2` is `True`.
4. Add a short explanation in a markdown cell summarizing the difference between identity and equality
   in this context.


In [9]:
# === Your work for Problem 5 goes here ===

class User:
    pass  # TODO: implement __init__, __repr__, and __eq__

# --- Demo / tests ---
# u1 = User("alice", "alice@example.com")
# u2 = User("alice", "alice@example.com")
# u3 = User("bob", "bob@example.com")

# print("u1:", u1)
# print("u2:", u2)
# print("u3:", u3)

# print("u1 is u2:", u1 is u2)   # False
# print("u1 == u2:", u1 == u2)   # True
# print("u1 == u3:", u1 == u3)   # False
# print("u1 == 'alice':", u1 == "alice")  # False


### Solution – Problem 5


In [10]:
class User:
    def __init__(self, username: str, email: str):
        self.username = username
        self.email = email

    def __repr__(self) -> str:
        return f"User(username={self.username!r}, email={self.email!r})"

    def __eq__(self, other) -> bool:
        if not isinstance(other, User):
            return NotImplemented
        # Business rule: same username => same logical user
        return self.username == other.username

# Demonstration
u1 = User("alice", "alice@example.com")
u2 = User("alice", "alice@example.com")
u3 = User("bob", "bob@example.com")

print("u1:", u1)
print("u2:", u2)
print("u3:", u3)

print("u1 is u2:", u1 is u2)   # different objects (different identities)
print("u1 == u2:", u1 == u2)   # logically equal under our rule
print("u1 == u3:", u1 == u3)
print("u1 == 'alice':", u1 == "alice")


u1: User(username='alice', email='alice@example.com')
u2: User(username='alice', email='alice@example.com')
u3: User(username='bob', email='bob@example.com')
u1 is u2: False
u1 == u2: True
u1 == u3: False
u1 == 'alice': False


### Identity vs Equality – Summary

* `is` compares object identity: `a is b` is `True` only if `a` and `b` are literally the same object
  (same memory address, same `id(a)`).
* `==` compares value equality: `a == b` calls `a.__eq__(b)` (or `b.__eq__(a)` if the first returns
  `NotImplemented`).

In Problem 5:

* `u1 is u2` is `False` because they are two distinct instances.
* `u1 == u2` is `True` because we defined equality only on the `username` attribute, so they represent
  the same logical user.
